In [ ]:
pip install pandas


In [ ]:
pip install pyspark

Here we install the dependancies that are pandas and sqlite3.   

Then we load the two csv files using the pandas function 'read_csv' and store them into two variables 
"deliveries" and "matches".

In [ ]:
import pandas as pd
import sqlite3

deliveries = pd.read_csv('deliveries.csv')
matches = pd.read_csv('matches.csv')

Here we are trying to connect to the SQLite database "cricket_data", and we store it in the variable "connect_db". In order to do that, we first use sqlite3's 'connect' function, that will check whether a file named "cricket_data.db" exists, if it's the case then it will connect to it; else it will create a new database file named like that. 

Then we create a cursor object in order to execute SQL commands to the database.

In [ ]:
connect_db = sqlite3.connect('cricket_data.db')
cursor = connect_db.cursor()

In [ ]:
deliveries.to_sql('deliveries', connect_db, if_exists='replace', index=False)
matches.to_sql('matches',connect_db,if_exists='replace',index=False)

In [ ]:
connect_db.commit()
# connect_db.close()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Créer une session Spark
spark = SparkSession.builder \
    .appName("Check NULLs in CSV files") \
    .getOrCreate()

# Charger les fichiers CSV dans des DataFrames Spark
deliveries_df = spark.read.csv('deliveries.csv', header=True, inferSchema=True)
matches_df = spark.read.csv('matches.csv', header=True, inferSchema=True)

# Fonction pour compter les valeurs NULL dans chaque colonne d'un DataFrame
def count_nulls(df):
    null_counts = {}
    for column in df.columns:
        # Compter les valeurs NULL
        null_count = df.filter(col(column).isNull()).count()
        
        # Stocker les résultats
        null_counts[column] = null_count
    return null_counts

# Compter les valeurs NULL pour le DataFrame deliveries
deliveries_null_counts = count_nulls(deliveries_df)

# Compter les valeurs NULL pour le DataFrame matches
matches_null_counts = count_nulls(matches_df)

# Afficher les résultats pour le fichier deliveries.csv
print("Résultats pour 'deliveries.csv':")
for column, count in deliveries_null_counts.items():
    print(f"Colonne '{column}': {count} NULL")

# Afficher les résultats pour le fichier matches.csv
print("\nRésultats pour 'matches.csv':")
for column, count in matches_null_counts.items():
    print(f"Colonne '{column}': {count} NULL")

# Fermer la session Spark
spark.stop()

In [ ]:
#Delete the columns with too many null values
matches_df = matches_df.drop('umpire3')

# Supprimer la colonne "player_dismissal" de deliveries_df
deliveries_df = deliveries_df.drop('player_dismissal').drop('dismissal_kind').drop('fielder')

# Afficher le schéma mis à jour des DataFrames pour vérification
print("Schema de deliveries_df après suppression de 'player_dismissal':")
deliveries_df.printSchema()

print("\nSchema de matches_df après suppression de 'umpire3':")
matches_df.printSchema()

# Fermer la session Spark
spark.stop()

In [ ]:
# Créer une session Spark
spark = SparkSession.builder \
    .appName("Display Venue when City is Null") \
    .getOrCreate()

# Charger le fichier CSV dans un DataFrame Spark
matches_df = spark.read.csv('matches.csv', header=True, inferSchema=True)

# Filtrer les lignes où la colonne 'city' est null
venues_with_null_city = matches_df.filter(matches_df['city'].isNull()).select('venue')

# Afficher les venues lorsque la city est null
print("Venues when city is null:")
venues_with_null_city.show(truncate=False)

# Fermer la session Spark
spark.stop()

In [ ]:
from pyspark.sql.functions import when

# Créer une session Spark
spark = SparkSession.builder \
    .appName("Replace City when Venue is Dubai International Cricket Stadium") \
    .getOrCreate()

# Charger le fichier CSV dans un DataFrame Spark
matches_df = spark.read.csv('matches.csv', header=True, inferSchema=True)

# Remplacer la valeur de 'city' lorsque 'venue' est 'Dubai International Cricket Stadium'
matches_df = matches_df.withColumn('city', when(matches_df['venue'] == 'Dubai International Cricket Stadium', 'Dubai').otherwise(matches_df['city']))

# Fermer la session Spark
spark.stop()


In [ ]:
# Create Spark session
spark = SparkSession.builder \
    .appName("Min and Max Check") \
    .getOrCreate()

# Load CSV data into DataFrames
matches_df = spark.read.csv('matches.csv', header=True, inferSchema=True)
deliveries_df = spark.read.csv('deliveries.csv', header=True, inferSchema=True)


from pyspark.sql.functions import min, max

# Specify columns to check
matches_columns_to_check = ["season", "dl_applied", "win_by_runs", "win_by_wickets"]

# Compute min and max for each column in matches
matches_min = matches_df.select([min(col).alias(f"min_{col}") for col in matches_columns_to_check])
matches_max = matches_df.select([max(col).alias(f"max_{col}") for col in matches_columns_to_check])

# Show the results
matches_min.show(truncate=False)
matches_max.show(truncate=False)



# Specify columns to check
deliveries_columns_to_check = ["match_id", "inning", "over", "ball", "is_super_over", "wide_runs", "bye_runs",
                               "legbye_runs", "noball_runs", "penalty_runs", "batsman_runs", "extra_runs", "total_runs"]

# Compute min and max for each column in deliveries
deliveries_min = deliveries_df.select([min(col).alias(f"min_{col}") for col in deliveries_columns_to_check])
deliveries_max = deliveries_df.select([max(col).alias(f"max_{col}") for col in deliveries_columns_to_check])

# Show the results
deliveries_min.show(truncate=False)
deliveries_max.show(truncate=False)
